# Deploys model to IBM Watson Machine Learning
Currently only supports TensorFlow V2.4 SavedModel format. More to come...

In [ ]:
!pip3 install ibm-watson-machine-learning wget==3.2

In [ ]:
import wget
wget.download(
    'https://raw.githubusercontent.com/'
    'elyra-ai/component-library/master/claimed_utils.py'
)

In [ ]:
from claimed_utils import unzip
from ibm_watson_machine_learning import APIClient
import json
import os

In [ ]:
# @dependency model.zip
# @dependency codait_utils.ipynb
# @param api_key IBM Cloud api_key
# @param space IBM Cloud deployment space ID
# @param location IBM Cloud location (e.g. us-south)
# @param model_zip file name
# @param deployment_name Deployment Name
# @param model_name Model Name

In [ ]:
api_key = os.environ.get('api_key')
space = os.environ.get('space')
location = os.environ.get('location', 'us-south')
model_zip = os.environ.get('model_zip', 'model.zip')
deployment_name = os.environ.get('deployment_name', 'Deployment Name')
model_name = os.environ.get('model_name', 'Model Name')

In [ ]:
wml_credentials = {
    "apikey": api_key,
    "url": 'https://' + location + '.ml.cloud.ibm.com'
}

In [ ]:
client = APIClient(wml_credentials)

In [ ]:
unzip('.', 'model.zip')

In [ ]:
!rm model.tar.gz

In [ ]:
%%bash
cd model
tar -czvf ../model.tar.gz *
cd ..

In [ ]:
!tar tvf model.tar.gz

In [ ]:
o = client.software_specifications.get_uid_by_name('tensorflow_2.4-py3.7')
software_spec_uid = o
client.set.default_space(space)

In [ ]:
model_meta_props = {
    client.repository.ModelMetaNames.NAME: deployment_name,
    client.repository.ModelMetaNames.TYPE: "tensorflow_2.4",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid
}

published_model = client.repository.store_model(
    model='model.tar.gz',
    meta_props=model_meta_props
)

model_uid = client.repository.get_model_uid(published_model)

In [ ]:
model_details = client.repository.get_details(model_uid)
print(json.dumps(model_details, indent=2))

In [ ]:
client.repository.list_models(limit=15)

In [ ]:
deployment = client.deployments.create(
    model_uid,
    meta_props={
        client.deployments.ConfigurationMetaNames.NAME: model_name,
        client.deployments.ConfigurationMetaNames.ONLINE: {}
    }
)

deployment_uid = client.deployments.get_uid(deployment)

In [ ]:
deployment_uid